# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [29]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import cv2


We set the random seed so that the results don't vary drastically.

In [30]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [31]:
train_doc = np.random.permutation(open('./notebooks/storage/Final_data/Collated_training/train.csv').readlines())
val_doc = np.random.permutation(open('./notebooks/storage/Final_data/Collated_training/val.csv').readlines())
batch_size = 64 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [32]:
width = 100
height = 100

In [33]:
def load_images(batch,batch_size,img_idx,source_path,t):
    batch_data = np.zeros((batch_size,len(img_idx),width,height,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    for folder in range(batch_size): # iterate over the batch_size
        imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
        for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
            image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes 
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if image.shape[0] != image.shape[1]:
                image=image[0:120,20:140]
            resized_image = imresize(image,(width,height))
            #norm_image = resized_image - np.percentile(resized_image,5)/ np.percentile(resized_image,95) - np.percentile(resized_image,5)
            norm_image= resized_image/255
            batch_data[folder,idx,:,:,0] = norm_image[:,:,0]
            batch_data[folder,idx,:,:,1] = norm_image[:,:,1]
            batch_data[folder,idx,:,:,2] = norm_image[:,:,2]
        batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
    return batch_data,batch_labels

In [34]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = np.arange(6,30,2) #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            yield load_images(batch,batch_size,img_idx,source_path,t) #you yield the batch_data and the batch_labels, remember what does yield do
        if (len(folder_list) != batch_size*num_batches):
            batch=+1
            reduced_batch_size = len(folder_list) - (batch_size*num_batches)
            yield load_images(batch,reduced_batch_size,img_idx,source_path,t)


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [35]:
curr_dt_time = datetime.datetime.now()
train_path = './notebooks/storage/Final_data/Collated_training/train'
val_path = './notebooks/storage/Final_data/Collated_training/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [22]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import l2
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, RNN

model = Sequential()

model.add(TimeDistributed(Conv2D(16, (2, 2), padding='same'),
                 input_shape=(12,width,height,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(TimeDistributed(Conv2D(16, (2, 2))))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))


model.add(TimeDistributed(Conv2D(32, (2, 2), padding='same')))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(TimeDistributed(Conv2D(32, (2, 2), padding='same')))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(Dropout(0.2))

model.add(TimeDistributed(Conv2D(64, (2, 2), padding='same')))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(TimeDistributed(Conv2D(64, (2, 2), padding='same')))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(Dropout(0.2))


model.add(TimeDistributed(Flatten()))
model.add(GRU(256, return_sequences=False, dropout=0.5))
model.add(Dense(64,kernel_regularizer=l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [37]:
# compile it
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['categorical_accuracy'])

# summary of model
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_11 (TimeDis (None, 12, 100, 100, 16)  208       
_________________________________________________________________
activation_9 (Activation)    (None, 12, 100, 100, 16)  0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 12, 100, 100, 16)  64        
_________________________________________________________________
time_distributed_12 (TimeDis (None, 12, 99, 99, 16)    1040      
_________________________________________________________________
activation_10 (Activation)   (None, 12, 99, 99, 16)    0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 12, 99, 99, 16)    64        
_________________________________________________________________
time_distributed_13 (TimeDis (None, 12, 49, 49, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [38]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [39]:
# model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
model_name = 'model' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

#LR = # write the REducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.01)
callbacks_list = [checkpoint]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [40]:

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [41]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./notebooks/storage/Final_data/Collated_training/val ; batch size = 64
Source path =  ./notebooks/storage/Final_data/Collated_training/train ; batch size = 64
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  import sys
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


11/11 [==============================] - 39s 4s/step - loss: 0.3006 - categorical_accuracy: 0.9929 - val_loss: 1.0772 - val_categorical_accuracy: 0.7200

Epoch 00001: saving model to model_2020-11-0909_29_03.691808/model-00001-0.30095-0.99246-1.07719-0.72000.h5
Epoch 2/20
11/11 [==============================] - 31s 3s/step - loss: 0.2988 - categorical_accuracy: 0.9957 - val_loss: 0.9654 - val_categorical_accuracy: 0.7600

Epoch 00002: saving model to model_2020-11-0909_29_03.691808/model-00002-0.29913-0.99548-0.96543-0.76000.h5
Epoch 3/20
11/11 [==============================] - 31s 3s/step - loss: 0.2957 - categorical_accuracy: 0.9943 - val_loss: 0.9626 - val_categorical_accuracy: 0.7300

Epoch 00003: saving model to model_2020-11-0909_29_03.691808/model-00003-0.29653-0.99397-0.96263-0.73000.h5
Epoch 4/20
11/11 [==============================] - 31s 3s/step - loss: 0.2941 - categorical_accuracy: 0.9943 - val_loss: 0.8918 - val_categorical_accuracy: 0.8200

Epoch 00004: saving model t